# Perform Multi-Table Synthesis

In this exercise, we are going to walk through the synthesis of a relational table structure. For that, we will be using a slightly trimmed down version of the Berka dataset [[1](#refs)]. It consists of a total of 8 tables, with one of these ("district") serving as a reference table, and all others containing privacy-sensitive information.

<img src='./berka-original.png' width="600px"/>

At the time of writing, MOSTLY AI requires data for a multi-table scenario the data to be provisioned via a relational database, and also to be delivered to a relational database. Thus we will first provide helper scripts for creating two public database instances, to load the original data into one of them, and to then make the required job configuration.

Once synthesized, we will check for referential integrity, as well as for the retention of specific statistical properties that span multiple tables.

## Import Data to a Database

If you don't have a DB server available, then go to your preferred cloud provider (AWS, GPC, Azure, etc.) and launch an instance there first. Make sure that clients can connect externally via username / password credentials, and have the required rights to create, update and delete database instances there.

<img src='./gpc1.png' width="400px"/> <img src='./gpc2.png' width="400px"/><br /><img src='./gpc3.png' width="400px"/> <img src='./gpc4.png' width="400px"/>

Once in place, please update the following variables accordingly.

In [71]:
db_host = "34.22.179.186"
db_usr = "postgres"
db_pwd = "berka"

Let's then create two database instances. One, that will contain the original data. And another one, that will serve us as a destination for the synthetic tables.

In [247]:
# install required Python packages
!pip install -q psycopg2 sqlalchemy

In [73]:
import sqlalchemy
assert sqlalchemy.__version__.startswith('2.')

### Create Source and Destination Database

In [75]:
import psycopg2
from sqlalchemy import create_engine

def create_db(host, user, pwd, db_name, if_exists="fail"):
    con = psycopg2.connect(f"postgresql://{user}:{pwd}@{host}:5432/postgres")
    con.autocommit = True
    cur = con.cursor()
    cur.execute(f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = '{db_name}'")
    exists = cur.fetchone()
    if exists and if_exists == "fail":
        raise Exception(f"database {db_name} already exists")
    elif exists and if_exists == "replace":
        cur.execute("DROP DATABASE " + db_name)
    cur.execute("CREATE DATABASE " + db_name)
    con.close()

db_name_source = 'berka_original'
db_name_destination = 'berka_synthetic'
create_db(db_host, db_usr, db_pwd, db_name_source, if_exists="replace")
create_db(db_host, db_usr, db_pwd, db_name_destination, if_exists="replace")

### Load Data into Source Database

In [76]:
def connect_db(host, user, pwd, db_name):
    engine = create_engine(f"postgresql://{user}:{pwd}@{host}:5432/{db_name}")
    return engine

engine = connect_db(db_host, db_usr, db_pwd, db_name_source)

In [77]:
# check whether we are in Google colab
try:
    from google.colab import files
    print("running in COLAB mode")
    repo = 'https://github.com/mostly-ai/mostly-tutorials/raw/dev/multi-table'
except:
    print("running in LOCAL mode")
    repo = '.'

running in LOCAL mode


In [78]:
# import data into DB
from pathlib import Path
import pandas as pd
csv_files = [
    f'{repo}/account.csv', 
    f'{repo}/card.csv', 
    f'{repo}/client.csv', 
    f'{repo}/disposition.csv', 
    f'{repo}/district.csv', 
    f'{repo}/loan.csv', 
    f'{repo}/orders.csv', 
    f'{repo}/transaction.csv'
]
originals = {}
for fn in csv_files:
    # read data from CSV into Pandas DataFrame
    df = pd.read_csv(fn)
    # ensure all columns are NULL-able
    df = df.convert_dtypes()
    # convert date columns
    for col in df.columns:
        if col in ['date', 'issued']:
            df[col] = pd.to_datetime(df[col])
    # get filename w/o extension
    db_table = Path(fn).stem
    # write DataFrame to DB
    df.to_sql(db_table, engine, index=False, if_exists='replace')
    print(f"created table `{db_table}` with {df.shape[0]:,} records")
    originals[db_table] = df
    
print('DONE')

created table `account` with 4,500 records
created table `card` with 892 records
created table `client` with 5,369 records
created table `disposition` with 5,369 records
created table `district` with 77 records
created table `loan` with 682 records
created table `orders` with 6,471 records
created table `transaction` with 49,498 records
DONE


In [79]:
from sqlalchemy import text
with engine.connect() as conn:
    # define primary keys in the database
    conn.execute(text('ALTER TABLE account ADD PRIMARY KEY (account_id);'))
    conn.execute(text('ALTER TABLE card ADD PRIMARY KEY (card_id);'))
    conn.execute(text('ALTER TABLE client ADD PRIMARY KEY (client_id);'))
    conn.execute(text('ALTER TABLE disposition ADD PRIMARY KEY (disp_id);'))
    conn.execute(text('ALTER TABLE district ADD PRIMARY KEY (district_id);'))
    conn.execute(text('ALTER TABLE loan ADD PRIMARY KEY (loan_id);'))
    conn.execute(text('ALTER TABLE orders ADD PRIMARY KEY (order_id);'))
    conn.execute(text('ALTER TABLE transaction ADD PRIMARY KEY (trans_id);'))
    print(f"created primary keys")
    # define foreign key constraints in the database
    conn.execute(text('ALTER TABLE account ADD CONSTRAINT fk_district_a FOREIGN KEY (district_id) REFERENCES district (district_id);'))
    conn.execute(text('ALTER TABLE client ADD CONSTRAINT fk_district_c FOREIGN KEY (district_id) REFERENCES district (district_id);'))
    conn.execute(text('ALTER TABLE disposition ADD CONSTRAINT fk_disp_a FOREIGN KEY (account_id) REFERENCES account (account_id);'))
    conn.execute(text('ALTER TABLE disposition ADD CONSTRAINT fk_disp_c FOREIGN KEY (client_id) REFERENCES client (client_id);'))
    conn.execute(text('ALTER TABLE card ADD CONSTRAINT fk_card FOREIGN KEY (disp_id) REFERENCES disposition (disp_id);'))
    conn.execute(text('ALTER TABLE transaction ADD CONSTRAINT fk_trans FOREIGN KEY (account_id) REFERENCES account (account_id);'))
    conn.execute(text('ALTER TABLE loan ADD CONSTRAINT fk_loan FOREIGN KEY (account_id) REFERENCES account (account_id);'))
    conn.execute(text('ALTER TABLE orders ADD CONSTRAINT fk_order FOREIGN KEY (account_id) REFERENCES account (account_id);'))
    print(f"created foreign keys")
    conn.commit()
print('DONE')

created primary keys for all tables
created foreign keys for all tables


## Synthesize Data via MOSTLY AI

Go to MOSTLY AI, and

1. Create two data connectors, one for the source DB `berka_original`, and one for the destination DB `berka_synthetic`

2. Create a data catalog using the data connector for `berka_original`

    - Select all 8 tables for the data catalog
    - Select `client` and `account` as subject tables
    - Keep the ranking of the subject table as-is
    - Configure smart select column `district_id` for the `disposition -> client` relation

<img src='./mostly1.png' width="400px"/> <img src='./mostly2.png' width="400px"/><br />
<img src='./mostly3.png' width="400px"/> <img src='./mostly4.png' width="400px"/><br />
<img src='./mostly5.png' width="400px"/> <img src='./mostly6.png' width="400px"/><br />

These are then the configured table types and relations.

<img src='./berka-synthetic.png' width="600px"/>

3. Launch the job, and select `berka_synthetic` as a destination in "Output settings"

4. Once the job has completed, continue with executing the next cell

In [83]:
# fetch synthetic data from destination database
engine = connect_db(db_host, db_usr, db_pwd, db_name_destination)
tables = [Path(fn).stem for fn in csv_files]
synthetics = {}
for db_table in tables:
    with engine.begin() as conn:
        df = pd.read_sql_query(sql=text(f'select * from {db_table};'), con=conn)
    print(f"extracted table {db_table} with {df.shape[0]:,} records")
    synthetics[db_table] = df

extracted table account with 4,500 records
extracted table card with 5,344 records
extracted table client with 5,369 records
extracted table disposition with 5,344 records
extracted table district with 77 records
extracted table loan with 4,500 records
extracted table orders with 7,488 records
extracted table transaction with 49,500 records


## Inspect Synthetic Data

### Show sample records for each table

In [217]:
for k in originals:
    display(synthetics[k].head(3))

,account_id,district_id,frequency,date
0,1,65,POPLATEK MESICNE,1997-12-06
1,2,38,POPLATEK MESICNE,1997-04-16
2,3,76,POPLATEK MESICNE,1995-08-04


,card_id,disp_id,type,issued
0,1,1,classic,1998-01-06
1,2,2,classic,1998-07-09
2,3,3,gold,1998-09-23


,client_id,birth_number,district_id
0,1,405106,30
1,2,501012,76
2,3,440602,54


,disp_id,client_id,account_id,type
0,1,838,1,OWNER
1,2,3342,2,OWNER
2,3,3506,3,OWNER


,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813


,loan_id,account_id,date,amount,duration,payments,status
0,1,1,1997-11-21,12792,48,2105,C
1,2,2,1998-01-21,399408,36,8798,C
2,3,3,1996-04-15,293996,36,3539,C


,order_id,account_id,bank_to,account_to,amount,k_symbol
0,1,1,KL,89814086,3433.0,SIPO
1,2,1,GH,52245426,5015.0,
2,3,1,ST,16042956,4302.0,POJISTNE


,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank
0,1,1,1997-12-08,PRIJEM,VKLAD,900.0,900.0,None,None
1,2,1,1998-01-09,PRIJEM,VKLAD,12007.0,14701.0,None,None
2,3,1,1998-01-17,PRIJEM,VKLAD,6700.0,18109.0,None,None


### Check basic statistics

The newly generated tables are statistically representative of the original.

In [239]:
display(synthetics['transaction']['amount'].quantile(q=[.1, .5, .9]))
display(originals['transaction']['amount'].quantile(q=[.1, .5, .9]))

0.1       46.2
0.5     3374.0
0.9    23071.0
Name: amount, dtype: float64

0.1       52.3
0.5     3452.0
0.9    23227.2
Name: amount, dtype: float64

In [242]:
display(synthetics['account']['date'].quantile(q=[.1, .5, .9]))
display(pd.to_datetime(originals['account']['date']).quantile(q=[.1, .5, .9]))

0.1   1993-05-25
0.5   1995-12-03
0.9   1997-07-05
Name: date, dtype: datetime64[ns]

0.1   1993-05-24
0.5   1996-01-02
0.9   1997-06-29
Name: date, dtype: datetime64[ns]

### Check referential integrity

The newly generated foreign keys are also present as primary keys in the connected tables.

In [243]:
assert synthetics['transaction']['account_id'].isin(synthetics['account']['account_id']).all()
assert synthetics['client']['district_id'].isin(synthetics['district']['district_id']).all()
assert synthetics['card']['disp_id'].isin(synthetics['disposition']['disp_id']).all()

### Check context relations

The cardinality of context FK relations is perfectly retained.

In [244]:
print('Orders per Account - Original')
display(synthetics['orders'].groupby('account_id').size().value_counts())
print('\nOrders per Account - Synthetic')
display(originals['orders'].groupby('account_id').size().value_counts())

Orders per Account - Original


1    2533
2    1229
3     514
4     165
5      59
dtype: int64


Orders per Account - Synthetic


1    2103
2     949
3     416
4     228
5      62
dtype: int64

In [245]:
print('Cards per Disposition - Original')
display(synthetics['card'].groupby('disp_id').size().value_counts())
print('\nCards per Disposition - Synthetic')
display(originals['card'].groupby('disp_id').size().value_counts())

Cards per Disposition - Original


1    5344
dtype: int64


Cards per Disposition - Synthetic


1    892
dtype: int64

### Check smart select relations

The cardinality of smart select FK relation is not retained, as these get randomly assigned.

In [246]:
print('Dispositions per Client - Original')
display(originals['disposition'].groupby('client_id').size().value_counts())
print('\nDispositions per Client - Synthetic')
display(synthetics['disposition'].groupby('client_id').size().value_counts())

Dispositions per Client - Original


1    5369
dtype: int64


Dispositions per Client - Synthetic


1    1961
2     951
3     336
4      86
5      14
6       5
7       3
8       1
dtype: int64

Some of the statistical relations between a child and its randomly assigned smart select parent can be retained, if corresponding smart select columns were configured. E.g. if smart select is properly configured, then the the share of cases where the `client` has the same `district_id` as the `account`, that she owns, should be similar.

In [224]:
def matching_districts(datasets):    
    df = datasets['disposition']
    df = df.loc[df.type=='OWNER']
    df = df.merge(
        datasets['client'], 
        on='client_id',
    ).merge(
        datasets['account'], 
        on='account_id',
    )
    return (df['district_id_x']==df['district_id_y']).mean()

print(f"Share of accounts and clients with identical district_id")
print(f"original:  {matching_districts(originals):4.0%}")
print(f"synthetic: {matching_districts(synthetics):4.0%}")

Share of accounts and clients with identical district_id
original:   91%
synthetic:   3%


## Conclusion

In this tutorial we have demonstrated how to synthesize a multi-table relational database. We have seen, that structure, statistics and referential integrity are perfectly retained. We have also seen, how to configure Smart Select, and its impact on retaining statistcs across non-context relations. But we have also seen, that there are limitations to what can be retained, in particular when it comes to the cardinality of smart select relations.

## References<a class="anchor" name="refs"></a>

1. https://data.world/lpetrocelli/czech-financial-dataset-real-anonymized-transactions